In [159]:
import json
import os
import csv
from typing import Callable

import pandas as pd
import requests

In [164]:
current_working_directory = os.getcwd()
print("Location of the notebook "+current_working_directory)
os.chdir('../')
project_root=os.getcwd()
print("Project root "+project_root)

Location of the notebook /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo/alivio
Project root /Users/yaminigotimukul/DataScience/Berekley/Semesters/Spring_2024/repo


In [166]:
DATA_DIR: str = project_root + "/data"
BASE_DIR: str = f"{DATA_DIR}/openai_caribbean/submission_github_data/train.json"

### Buildging Damage Helper Functions

In [167]:
def convert_json_to_csv(path, output_name):
    json_files = [pos_json for pos_json in os.listdir(path) if pos_json.endswith('.json')]

    count = 0
    count1 = 0
    for index, js in enumerate(json_files):
        with open(os.path.join(path, js)) as json_file:
            data_mx_eq = json.load(json_file)
        features = data_mx_eq["features"]

        data_file = open('train_'+output_name+'_geo_details.csv', 'a')
        csv_writer = csv.writer(data_file)


        data_file1 = open('train_'+output_name+'_geo_xy_details.csv', 'a')
        csv_writer1 = csv.writer(data_file1)



        for lng_lat in features['lng_lat']:

            if count == 0:
                header = ['feature_type', 'damage_level', 'uid', 'wkt_lt_lg', 'lat_lng',"img_name","sensor","provider_asset_type", "gsd", \
                          "capture_date", "off_nadir_angle", "pan_resolution", "sun_azimuth", "sun_elevation", \
                          "target_azimuth", "disaster", "disaster_type", "catalog_id", "original_width", "original_height" \
                         "width", "height", "metadata_id" ]
                csv_writer.writerow(header)
                print(data_mx_eq['metadata']['gsd'])
                #print(lng_lat['properties']['feature_type'])
                count+=1

            csv_writer.writerow([lng_lat['properties']['feature_type'], \
                                 lng_lat['properties']['subtype'], \
                                 lng_lat['properties']['uid'], \
                                 lng_lat['wkt'], \
                                 lng_lat['properties'], \
                                 data_mx_eq['metadata']['img_name'], \
                                 data_mx_eq['metadata']['sensor'], \
                                 data_mx_eq['metadata']['provider_asset_type'], \
                                 str(data_mx_eq['metadata']['gsd']), \
                                 data_mx_eq['metadata']['capture_date'], \
                                 str(data_mx_eq['metadata']['off_nadir_angle']), \
                                 str(data_mx_eq['metadata']['pan_resolution']), \
                                 str(data_mx_eq['metadata']['sun_azimuth']), \
                                 str(data_mx_eq['metadata']['sun_elevation']), \
                                 str(data_mx_eq['metadata']['target_azimuth']), \
                                 data_mx_eq['metadata']['disaster'], \
                                 data_mx_eq['metadata']['disaster_type'], \
                                 data_mx_eq['metadata']['catalog_id'], \
                                 str(data_mx_eq['metadata']['original_width']), \
                                 str(data_mx_eq['metadata']['original_height']), \
                                 str(data_mx_eq['metadata']['width']), \
                                 str(data_mx_eq['metadata']['height']), \
                                 data_mx_eq['metadata']['id']
                                ]
                               )

        for xy in features['xy']:    
            if count1 == 0:
                #print(xy['properties']['feature_type'])
                header1= ["uid","wkt","wkt_xy"]
                csv_writer1.writerow(header1)
                count1+=1

            csv_writer1.writerow([xy['properties']['uid'],xy['wkt'],xy['properties']])
            
        
    

In [152]:
def merge_geo_output_files(output_name):    
        df_f1=pd.read_csv("train_"+output_name+"_geo_details.csv", index_col= False)
        df_f2=pd.read_csv("train_"+output_name+"_geo_xy_details.csv", index_col= False)

        df_f1.uid=df_f1.uid.astype(str)
        df_f2.uid=df_f2.uid.astype(str)

        df_merged = pd.merge(df_f1, df_f2, on='uid')
        df_merged.to_csv('train_'+output_name+'.csv', index=False)

## Data Conversion

### Roof Type Classification

In [10]:
with open(BASE_DIR) as json_file:
    data = json.load(json_file)

features=data["features"]
data_file = open('train_roof_types_geo_details.csv', 'w')
csv_writer = csv.writer(data_file)

count = 0

for feature in features:
    if count == 0:
 
        # Writing headers of CSV file
        header = ["type", "id","roof_material", "verified", "name", "geometry_type", "cordinates", "geometry"]
        csv_writer.writerow(header)
        count += 1
 
    # Writing data of CSV file

    csv_writer.writerow([feature["type"],feature["properties"]["id"],feature["properties"]["roof_material"], feature["properties"]["verified"],feature["properties"]["name"], feature["geometry"]["type"], feature["geometry"]["coordinates"], feature["geometry"]] )


data_file.close()

### XView Building Damage Dataset

In [5]:
BDD_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/xview_geotransforms.json"

In [6]:
with open(BDD_BASE_DIR) as json_file:
    bdd_data = json.load(json_file)

In [18]:
keys=bdd_data.keys()

In [32]:
data_file = open('train_roof_damage_geo_details.csv', 'w')
csv_writer = csv.writer(data_file)

count = 0

for key in keys:
    if count == 0:
 
        # Writing headers of CSV file
        header = ["image_id", "location","GEOGCS"]
        csv_writer.writerow(header)
        count += 1
 
    # Writing data of CSV file

    csv_writer.writerow([key,bdd_data[key][0],bdd_data[key][1]])


data_file.close()

## Mexico Earthquake - Damage datasets

In [37]:
MX_EQ_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/mexico/post_disaster/"

In [40]:
json_files = [pos_json for pos_json in os.listdir(MX_EQ_BASE_DIR) if pos_json.endswith('.json')]
json_files[:3]

['mexico-earthquake_00000164_post_disaster.json',
 'mexico-earthquake_00000107_post_disaster.json',
 'mexico-earthquake_00000032_post_disaster.json']

In [137]:
df_f1=pd.read_csv("train_mx_eq_geo_details.csv", index_col= False)
df_f2=pd.read_csv("train_mx_eq_geo_xy_details.csv", index_col= False)

/var/folders/_n/997tnw7n3tdg9z81cr8j96dc0000gn/T/ipykernel_89436/1136020967.py:1: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_f1=pd.read_csv("train_mx_eq_geo_details.csv", index_col= False)


In [138]:
df_f1.uid=df_f1.uid.astype(str)
df_f2.uid=df_f2.uid.astype(str)

In [139]:
df_f1

,feature_type,damage_level,uid,wkt_lt_lg,lat_lng,img_name,sensor,provider_asset_type,gsd,capture_date,...,sun_azimuth,sun_elevation,target_azimuth,disaster,disaster_type,catalog_id,original_width,original_heightwidth,height,metadata_id
0,building,no-damage,28119d94-b86c-47aa-bf75-6c42a368ed0c,POLYGON ((-99.15308685010807 19.25984242069931...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000164_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,148.53,68.674,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024
1,building,no-damage,50b3ec47-afef-441a-9145-330edfb641d9,POLYGON ((-99.15304572138677 19.26399182582984...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000164_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,148.53,68.674,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024
2,building,no-damage,7c94a152-05cb-4d2e-9a99-e827e4538046,POLYGON ((-99.15313713712854 19.26389313991497...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000164_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,148.53,68.674,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024
3,building,no-damage,37c80881-34c7-4970-8dfe-cd2bfd918034,POLYGON ((-99.15362687007375 19.26403145735543...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000164_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,148.53,68.674,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024
4,building,no-damage,4b0a2020-ecf2-420e-8499-97df109b7e2c,POLYGON ((-99.15378920194375 19.26417265095659...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000164_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,148.53,68.674,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32263,building,no-damage,0ebd7459-a529-4ad1-829b-02730f8652f5,"POLYGON ((-99.1630583608366 19.29549340808691,...","{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000126_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,148.53,68.674,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024
32264,building,no-damage,de92c7b4-281a-4034-a070-f37c16b9ab74,POLYGON ((-99.16308287462437 19.29529677867995...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000126_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,148.53,68.674,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024
32265,building,no-damage,5ba086bd-dfbd-461d-a4f6-4582d204ad98,POLYGON ((-99.16268003907992 19.29555725941927...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000126_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,148.53,68.674,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024
32266,building,no-damage,e8d53af5-d795-4b2d-9af3-1ad02e2de5df,POLYGON ((-99.16251399311143 19.29559900916098...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000126_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,148.53,68.674,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024


In [168]:
df_mexico1 = pd.merge(df_f1, df_f2, on='uid')

df_mexico1.head()

,feature_type,damage_level,uid,wkt_lt_lg,lat_lng,img_name,sensor,provider_asset_type,gsd,capture_date,...,target_azimuth,disaster,disaster_type,catalog_id,original_width,original_heightwidth,height,metadata_id,wkt,wkt_xy
0,building,no-damage,28119d94-b86c-47aa-bf75-6c42a368ed0c,POLYGON ((-99.15308685010807 19.25984242069931...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000164_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024,"POLYGON ((1013.214492098976 1022.976184481574,...","{'feature_type': 'building', 'subtype': 'no-da..."
1,building,no-damage,50b3ec47-afef-441a-9145-330edfb641d9,POLYGON ((-99.15304572138677 19.26399182582984...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000164_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024,"POLYGON ((1022.975832794109 74.78276414569109,...","{'feature_type': 'building', 'subtype': 'no-da..."
2,building,no-damage,7c94a152-05cb-4d2e-9a99-e827e4538046,POLYGON ((-99.15313713712854 19.26389313991497...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000164_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024,"POLYGON ((1003.117292562625 97.31649192069297,...","{'feature_type': 'building', 'subtype': 'no-da..."
3,building,no-damage,37c80881-34c7-4970-8dfe-cd2bfd918034,POLYGON ((-99.15362687007375 19.26403145735543...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000164_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024,"POLYGON ((896.8654319496612 65.61500846844616,...","{'feature_type': 'building', 'subtype': 'no-da..."
4,building,no-damage,4b0a2020-ecf2-420e-8499-97df109b7e2c,POLYGON ((-99.15378920194375 19.26417265095659...,"{'feature_type': 'building', 'subtype': 'no-da...",mexico-earthquake_00000164_post_disaster.png,WORLDVIEW02,WORLDVIEW02,2.650321,2017-09-20T17:46:11.000Z,...,105.50913,mexico-earthquake,earthquake,1030010072469D00,1024,1024,1024,1024,"POLYGON ((861.6654414518277 33.31896470811938,...","{'feature_type': 'building', 'subtype': 'no-da..."


In [142]:
df_mexico1.to_csv('train_mexico_eq_blds_damage.csv', index=False)

## Hurricane Florence - damage

In [154]:
HC_FL_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/hurricane/florence/post_disaster/"
convert_json_to_csv(HC_FL_BASE_DIR,"huricane_florence")    
    

2.0916247


In [155]:
merge_geo_output_files("huricane_florence")

/var/folders/_n/997tnw7n3tdg9z81cr8j96dc0000gn/T/ipykernel_89436/2411827772.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_f1=pd.read_csv("train_"+output_name+"_geo_details.csv", index_col= False)


In [156]:
pd.read_csv('train_huricane_florence.csv').head()

,feature_type,damage_level,uid,wkt_lt_lg,lat_lng,img_name,sensor,provider_asset_type,gsd,capture_date,...,target_azimuth,disaster,disaster_type,catalog_id,original_width,original_heightwidth,height,metadata_id,wkt,wkt_xy
0,building,minor-damage,61e6c19b-8adf-4f85-ab44-048b53395143,"POLYGON ((-77.90764031921604 34.8103647423034,...","{'feature_type': 'building', 'subtype': 'minor...",hurricane-florence_00000538_post_disaster.png,GEOEYE01,GEOEYE01,2.091625,2018-09-20T16:04:41.000Z,...,190.82309,hurricane-florence,flooding,1050010012411600,1024,1024,1024,1024,"POLYGON ((92.99243866758681 826.8137930208385,...","{'feature_type': 'building', 'subtype': 'minor..."
1,building,un-classified,bf2b217e-2c69-454e-af1d-2abe6265e3cd,POLYGON ((-77.90808191579313 34.80973024327165...,"{'feature_type': 'building', 'subtype': 'un-cl...",hurricane-florence_00000538_post_disaster.png,GEOEYE01,GEOEYE01,2.091625,2018-09-20T16:04:41.000Z,...,190.82309,hurricane-florence,flooding,1050010012411600,1024,1024,1024,1024,POLYGON ((-8.769107114089246e-06 978.457033399...,"{'feature_type': 'building', 'subtype': 'un-cl..."
2,building,un-classified,debd9427-5951-4882-9757-046aee1652de,POLYGON ((-77.90599617852514 34.81234820561588...,"{'feature_type': 'building', 'subtype': 'un-cl...",hurricane-florence_00000538_post_disaster.png,GEOEYE01,GEOEYE01,2.091625,2018-09-20T16:04:41.000Z,...,190.82309,hurricane-florence,flooding,1050010012411600,1024,1024,1024,1024,"POLYGON ((436.5393400504175 354.3019833824924,...","{'feature_type': 'building', 'subtype': 'un-cl..."
3,building,major-damage,339288fc-f2ac-47e0-917a-4e4c2b882ad0,"POLYGON ((-77.93204432743376 34.7115455171975,...","{'feature_type': 'building', 'subtype': 'major...",hurricane-florence_00000252_post_disaster.png,GEOEYE01,GEOEYE01,2.091625,2018-09-20T16:04:41.000Z,...,190.82309,hurricane-florence,flooding,1050010012411600,1024,1024,1024,1024,"POLYGON ((872.3163285131064 409.5181644413699,...","{'feature_type': 'building', 'subtype': 'major..."
4,building,un-classified,3bd12171-3421-4727-aa3d-612f278c7ac0,POLYGON ((-78.95302080681131 33.65446865832116...,"{'feature_type': 'building', 'subtype': 'un-cl...",hurricane-florence_00000028_post_disaster.png,GEOEYE01,GEOEYE01,2.895928,2018-09-18T16:32:47.000Z,...,69.02214,hurricane-florence,flooding,10500100123A5000,1024,1024,1024,1024,"POLYGON ((3.87645987039575 932.0996923560847, ...","{'feature_type': 'building', 'subtype': 'un-cl..."


## Hurricane Harvey - damage

In [157]:
HC_FL_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/hurricane/harvey/post_disaster/"
convert_json_to_csv(HC_FL_BASE_DIR,"huricane_harvey")   
merge_geo_output_files("huricane_harvey")

3.024613


/var/folders/_n/997tnw7n3tdg9z81cr8j96dc0000gn/T/ipykernel_89436/2411827772.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_f1=pd.read_csv("train_"+output_name+"_geo_details.csv", index_col= False)


## Hurricane Matthew - damage

In [158]:
HC_MW_BASE_DIR: str = f"{DATA_DIR}/xview_building_damage/hurricane/matthew/post_disaster/"
convert_json_to_csv(HC_MW_BASE_DIR,"huricane_matthew")   
merge_geo_output_files("huricane_matthew")

2.37887692


/var/folders/_n/997tnw7n3tdg9z81cr8j96dc0000gn/T/ipykernel_89436/2411827772.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df_f1=pd.read_csv("train_"+output_name+"_geo_details.csv", index_col= False)
